# Set up OAuth APIs with APImetrics KMS / JWT APIs
Running this script will create the APIs so we can sign JWTs and use the private_key_jwt authorization methods

## Requirements
1. Create an APImetrics Project
1. Get an API key with EDITOR permissions for the project, enter it below
1. Run this workbook to create all Auth Settings, APIs and Workflows for Sign-in flow
1. Set environment variables:
    1. kid_cert_version - this is the numeric ID internal to APImetrics for your signing key
    1. kid - this is your signing certs real key ID (as per JWKS)
    1. client_id - this is your client_id 
    1. issuer - this is the issuer for your JWTs
    1. redirect_uri - this is your redirect_uri for OAuth
    
NB: Until we publish the API call conditions API, those need to be added manually from the conditions tab

In [ ]:
# Enter intended project's APImetrics API key here:
API_KEY = input("Enter your APImetrics API key for the intended project: ")

In [ ]:
# Put your .well-known JSON here
true = True # to make copy-pasteing easier 
false = False # to make copy-pasteing easier 
WELL_KNOWN = None
if not WELL_KNOWN:
    WELL_KNOWN = json.loads(input("Enter your .well-known JSON here: "))

In [ ]:
# Enter your intended scope for your tokens
scopes_in = input("Enter the scopes you wish to use (except openid, we'll add that automatically): ")
scopes = ['openid'] + [scope.strip() for scope in scopes_in.split(' ')]
for scope in scopes:
    assert scope in WELL_KNOWN['scopes_supported'], f"Scope {scope} is not supported according to .well-known"
SCOPES = " ".join(scopes)
print(f"Using scopes {SCOPES}")

In [ ]:
assert 'private_key_jwt' in WELL_KNOWN['token_endpoint_auth_methods_supported'], 'This script is for private_key_jwt but your well-known does not claim that it is supported'

# We now extract the values we need (or you can override it)
AUTHORIZATION_URL = WELL_KNOWN['authorization_endpoint']
ACCESS_TOKEN_URL = WELL_KNOWN['token_endpoint']

In [ ]:
# Helper functions
import requests
import json
import urllib
from apimetrics_api import APImetricsAPI 
    
# An instance of the class that calls the APImetrics API
CLIENT = APImetricsAPI(API_KEY)

# [CLIENT.delete_token(o['id']) for o in CLIENT.tokens.values()]
# [CLIENT.delete_auth(o['id']) for o in CLIENT.auths.values()]
# [CLIENT.delete_call(o['id']) for o in CLIENT.calls.values()]
# [CLIENT.delete_workflow(o['id']) for o in CLIENT.workflows.values()]

From here on out, you can pick and choose what you want to run.


## Create Auth Setting for APImetrics API

In [ ]:
# First, create APImetrics API Auth Setting
tag = 'auth:apimetrics_api'
if tag not in CLIENT.auths_by_tag:
    setup = {
        "meta": {
            "domain": "client.apimetrics.io",
            "documentation": {
                "keys": "https://client.apimetrics.io/settings/api-key",
                "docs": "https://apimetrics.readme.io/v2/reference",
                "apps": "",
                "provider": "https://client.apimetrics.io/",
            },
            "name": "APImetrics API",
            "tags": [tag],
            "description": "API that allows you to call APImetrics' API.",
        },
        "settings": {"auth_type": "MANUAL"}
    }
    auth = CLIENT.create_auth(setup)
    print(f"Created Auth Setting {auth['meta']['name']} with id {auth['id']}")
else:
    print(f"Found Auth Setting {CLIENT.auths_by_tag[tag]}")

# Next, Create Token
if CLIENT.auths_by_tag[tag] not in CLIENT.tokens_by_auth:
    setup = {
        'meta': {
            'name': 'Project Access Token',
            'domain': 'client.apimetrics.io',
            'auth_id': CLIENT.auths_by_tag[tag]
        },
        'token': {
            'headers': [
              {
                'p_key': 'Authorization',
                'p_val': f'Bearer {API_KEY}',
              },
            ],
        }
    }
    token = CLIENT.create_token(setup)
    print(f"Created Auth Token {token['meta']['name']} with id {token['id']}")

## Create APImetrics API helpers for setting env variables

In [ ]:
def get_set_env(tag, var_name, value):
    body = {"value": value}
    body_str = json.dumps(body)

    setup = {
        "meta": {
            "description": None,
            "tags": ["api_type:update", "sector:devtools", tag],
            "name": "APImetrics: Save {}".format(var_name),
            "workspace": "global",
        },
        "request": {
            "body": body_str,
            "parameters": [],
            "url": "https://client.apimetrics.io/api/2/environment/global/{}".format(
                var_name
            ),
            "auth_id": CLIENT.auths_by_tag['auth:apimetrics_api'],
            "headers": [
                {"value": "application/json", "key": "Accept"},
                {"value": "application/json", "key": "Content-Type"},
            ],
            "token_id": CLIENT.tokens_by_auth[CLIENT.auths_by_tag['auth:apimetrics_api']],
            "method": "POST",
        },
    }
    return setup

In [ ]:
tag = 'apim:set_env:code'
if tag not in CLIENT.calls_by_tag:
    setup = get_set_env(tag, 'code', '__CODE__')
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

In [ ]:
tag = 'apim:set_env:refresh_token'
if tag not in CLIENT.calls_by_tag:
    setup = get_set_env(tag, 'refresh_token', '__REFRESH_TOKEN__')
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

## Create APImetrics API helper to save token

In [ ]:
tag = 'apim:tokens:update'
if tag not in CLIENT.calls_by_tag:
    body = {
        "token": {
            "headers": [
                {"p_key": "Authorization", "p_val": "Bearer %%ACCESS_TOKEN%%"},
            ],
            "expires_in": "%%EXPIRES_IN%%",
        }
    }
    body_str = json.dumps(body, indent=2)
    body_str = body_str.replace('"%%EXPIRES_IN%%"', "%%EXPIRES_IN%%")

    setup = {
        "meta": {
            "tags": ["api_type:update", "sector:devtools", tag],
            "name": "APImetrics: Update Auth Token",
            "workspace": "global",
        },
        "request": {
            "body": body_str,
            "parameters": [],
            "url": "https://client.apimetrics.io/api/2/tokens/{{apim_token_id}}/",
            "auth_id": CLIENT.auths_by_tag['auth:apimetrics_api'],
            "headers": [{"value": "application/json", "key": "Content-Type"}],
            "token_id": CLIENT.tokens_by_auth[CLIENT.auths_by_tag['auth:apimetrics_api']],
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")
    
    # Set environment variable we use in the API
    CLIENT.set_env_variable(
        'global', 
        'apim_token_id', 
        CLIENT.tokens_by_auth[CLIENT.auths_by_tag['auth:bank_matls'])

## APImetrics Helper APIs - Sign JWTs
### OAuth Assertions - Build JWT and Sign it

In [ ]:
def sign_empty_jwt(tag, name):
    auth_tag = 'auth:apimetrics_api'
    auth_id = CLIENT.auths_by_tag[auth_tag]
    token_id = CLIENT.tokens_by_auth[auth_id]
    
    body = {
        "header": {},
        "payload": {},
        "private_key": {
            "versionId": '{{kid_cert_version}}'
        }
    }
    body_str = json.dumps(body, indent=2)
    
    setup = {
        "meta": {
            "tags": ["api_type:create", "sector:devtools", tag],
            "name": f"Sign {name} JWT with KMS Cert",
            "workspace": "global",
        },
        "request": {
            "body": body_str,
            "parameters": [
                {"key": "kid", "value": "{{kid}}"},
                {"key": "alg", "value": "RS256"},
                {"key": "sub", "value": "{{client_id}}"},
                {"key": "aud", "value": "{{issuer}}"},
                {"key": "iss", "value": "{{client_id}}"},
                {"key": "expiresIn", "value": "1m"},
                {"key": "jti", "value": "__GUID__"},
                {"key": "notBefore", "value": "1"},
            ],
            "url": "https://us-central1-viatests.cloudfunctions.net/jwt-kms-signer",
            "auth_id": auth_id,
            "headers": [
                {"key": "Accept", "value": "application/json"},
                {"key": "Content-Type", "value": "application/json"},
            ],
            "token_id": token_id,
            "method": "POST",
        },
    }
    return setup

# For client credentials flow
tag = 'jwt:sign:client_credentials'
if tag not in CLIENT.calls_by_tag:
    setup = sign_empty_jwt(tag, 'Client Credentials')
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

# For authorization code flow
tag = 'jwt:sign:code'
if tag not in CLIENT.calls_by_tag:
    setup = sign_empty_jwt(tag, 'Auth Code')
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")
    
# For refresh token flow
tag = 'jwt:sign:refresh_token'
if tag not in CLIENT.calls_by_tag:
    setup = sign_empty_jwt(tag, 'Refresh Token')
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

### OAuth Authorize Claims - build JWT and sign it

In [ ]:
# For user authentication
tag = 'jwt:sign:authorize'
if tag not in CLIENT.calls_by_tag:
    auth_tag = 'auth:apimetrics_api'
    auth_id = CLIENT.auths_by_tag[auth_tag]
    token_id = CLIENT.tokens_by_auth[auth_id]
    
    body = {
        "header": {},
        "payload": {
            "scope": SCOPES,
            "claims": {
                "id_token": {
                    "acr": {"value": "urn:openbanking:psd2:sca", "essential": True},
                    "openbanking_intent_id": {
                        "value": "%%CONSENT_ID%%",
                        "essential": True,
                    },
                },
                "userinfo": {
                    "openbanking_intent_id": {
                        "value": "%%CONSENT_ID%%",
                        "essential": True,
                    }
                },
            },
            "response_type": "code id_token",
            "redirect_uri": "{{redirect_uri}}",
            "state": "__TEST_RUN_RESULT_ID__",
            "nonce": "__TEST_RUN_RESULT_ID__",
            "client_id": "{{client_id}}",
        },
        "private_key": {
            "versionId": '{{kid_cert_version}}'
        }
    }
    
    body_str = json.dumps(body, indent=2)
    
    setup = {
        "meta": {
            "tags": ["api_type:create", "sector:devtools", tag],
            "name": "Sign User Auth JWT with KMS Cert",
            "workspace": "global",
        },
        "request": {
            "body": body_str,
            "parameters": [
                {"value": "{{kid}}", "key": "kid"},
                {"value": "RS256", "key": "alg"},
                {"value": "{{issuer}}", "key": "aud"},
                {"value": "{{client_id}}", "key": "iss"},
                {"value": "10m", "key": "expiresIn"},
            ],
            "url": "https://us-central1-viatests.cloudfunctions.net/jwt-kms-signer",
            "auth_id": auth_id,
            "headers": [
                {"key": "Accept", "value": "application/json"},
                {"key": "Content-Type", "value": "application/json"},
            ],
            "token_id": token_id,
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

## OAuth - API calls
### Client Credentials flow

In [ ]:
tag = 'banks:oauth:client_credentials'
if tag not in CLIENT.calls_by_tag:
    
    assert 'auth:bank_matls' in CLIENT.auths_by_tag, 'Did not find a maTLS Auth Setting to use'
    
    headers = [{"key": "Content-Type", "value": "application/x-www-form-urlencoded"}]
    
    params = {
        "grant_type": "client_credentials",
        "scope": SCOPES,
        "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
        "client_assertion": "__JWT_TOKEN__",
    }
    params_str = urllib.parse.urlencode(params)
    
    setup = {
        "meta": {
            "description": "",
            "tags": ["api_type:create", "sector:financial", "ob_id:0", tag],
            "name": "OAuth: Access Token: client_credentials",
            "workspace": "global",
        },
        "request": {
            "body": params_str,
            "parameters": [],
            "url": ACCESS_TOKEN_URL,
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": headers,
            "token_id": None,
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

### Auth Code flow - go to user authorization page

In [ ]:
tag = 'banks:oauth:authorize'
if tag not in CLIENT.calls_by_tag:
    
    headers = [{"value": "*/*", "key": "Accept"}]
    
    params = {
        "response_type": "code id_token",
        "scope": SCOPES,
        "client_id": "{{client_id}}",
        "state": "__TEST_RUN_RESULT_ID__",
        "nonce": "__TEST_RUN_RESULT_ID__",
        "redirect_uri": "{{redirect_uri}}",
        "request": "__JWT_TOKEN__",
    }
    
    params_str = urllib.parse.urlencode(params)

    setup = {
        "meta": {
            "description": "",
            "tags": ["api_type:create", "sector:financial", "ob_id:0", tag],
            "name": "OAuth: Authorize",
            "workspace": "global",
        },
        "request": {
            "body": params_str,
            "parameters": [],
            "url": AUTHORIZATION_URL,
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": headers,
            "token_id": None,
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

### Auth Code flow - get access token

This is the call that swaps the temporary code from the user sign in redirect URL for an access token

In [ ]:
# For auth_code flow
tag = 'banks:oauth:code'
if tag not in CLIENT.calls_by_tag:
    assert 'auth:bank_matls' in CLIENT.auths_by_tag, 'Did not find a maTLS Auth Setting to use'
    
    headers = [{"key": "Content-Type", "value": "application/x-www-form-urlencoded"}]

    params = {
        "grant_type": "authorization_code",
        "code": "{{code}}",
        "redirect_uri": "{{redirect_uri}}",
        "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
        "client_assertion": "__JWT_TOKEN__",
    }
    params_str = urllib.parse.urlencode(params)

    setup = {
        "meta": {
            "description": "",
            "tags": ["api_type:create", "sector:financial", "ob_id:0", tag],
            "name": "OAuth: Access Token: authorization_code",
            "workspace": "global",
        },
        "request": {
            "body": params_str,
            "parameters": [],
            "url": ACCESS_TOKEN_URL,
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": headers,
            "token_id": None,
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

### Refresh token flow

In [ ]:
# For auth_code flow
tag = 'banks:oauth:refresh_token'
if tag not in CLIENT.calls_by_tag:
    assert 'auth:bank_matls' in CLIENT.auths_by_tag, 'Did not find a maTLS Auth Setting to use'
    
    headers = [{"key": "Content-Type", "value": "application/x-www-form-urlencoded"}]

    params = {
        "grant_type": "refresh_token",
        "refresh_token": "{{refresh_token}}",
        "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
        "client_assertion": "__JWT_TOKEN__",
    }
    params_str = urllib.parse.urlencode(params)

    setup = {
        "meta": {
            "description": "",
            "tags": ["api_type:create", "sector:financial", "ob_id:0", tag],
            "name": "OAuth: Access Token: refresh_token",
            "workspace": "global",
        },
        "request": {
            "body": params_str,
            "parameters": [],
            "url": ACCESS_TOKEN_URL,
            "auth_id": CLIENT.auths_by_tag['auth:bank_matls'],
            "headers": headers,
            "token_id": None,
            "method": "POST",
        },
    }
    call = CLIENT.create_call(setup)
    print(f"Created Call {call['meta']['name']} with id {call['id']}")

## Create Workflows to for User sign-in (authorize_code flow)

In [ ]:
# The user sign-in flow step 1
tag = 'banks:oauth:flow:authorize_code:1'
if tag not in CLIENT.workflows_by_tag:
            
    # This is the API that generates the claims you're going to use in the Auth
    claims_tag = 'banks:3.1:account-access-consents:create_max'
    
    call_tags = [
        'jwt:sign:client_credentials',
        'banks:oauth:client_credentials',
        claims_tag,
        'jwt:sign:authorize',
        'banks:oauth:authorize',
    ]
    for t in call_tags:
        assert t in CLIENT.calls_by_tag, f"API {t} does not exist"
    
    setup = {
      "meta": {
        "name": '[Manual] OAuth: Sign in as User (1/2)', 
        "workspace": "global", 
        "tags": [tag], 
      }, 
      "workflow": {
        "handle_cookies": False,
        "stop_on_failure": True,
        "call_ids": [CLIENT.calls_by_tag[t] for t in call_tags]
      }
    }
    call = CLIENT.create_workflow(setup)
    print(f"Created Workflow {call['meta']['name']} with id {call['id']}")


In [ ]:
# The user sign-in flow step 2
tag = 'banks:oauth:flow:authorize_code:2'
if tag not in CLIENT.workflows_by_tag:
            
    call_tags = [
        'apim:set_env:code',  # Save OAuth Code
        'jwt:sign:code',
        'banks:oauth:code',
        'apim:set_env:refresh_token',
        'apim:tokens:update',
    ]
    for t in call_tags:
        assert t in CLIENT.calls_by_tag, f"API {t} does not exist"
    
    setup = {
      "meta": {
        "name": "[Manual] OAuth: Sign in as User (2/2)", 
        "workspace": "global", 
        "tags": [tag], 
      }, 
      "workflow": {
        "handle_cookies": False,
        "stop_on_failure": True,
        "call_ids": [CLIENT.calls_by_tag[t] for t in call_tags]
      }
    }
    call = CLIENT.create_workflow(setup)
    print(f"Created Workflow {call['meta']['name']} with id {call['id']}")


## Create Refresh token workflow

In [ ]:
# A workflow to refresh our access token
tag = 'banks:oauth:flow:refresh_token'
if tag not in CLIENT.workflows_by_tag:
    call_tags = [
        'jwt:sign:client_credentials',
        'banks:oauth:refresh_token',
        'apim:tokens:update',
        'apim:set_env:refresh_token',
    ]
    for t in call_tags:
        assert t in CLIENT.calls_by_tag, f"API {t} does not exist"

    setup = {
      "meta": {
        "name": "OAuth: Refresh token", 
        "workspace": "global", 
        "tags": [tag], 
      }, 
      "workflow": {
        "handle_cookies": False,
        "stop_on_failure": True,
        "call_ids": [CLIENT.calls_by_tag[t] for t in call_tags]
      }
    }
    workflow = CLIENT.create_workflow(setup)
    print(f"Created Workflow {workflow['meta']['name']} with id {workflow['id']}")